In [ ]:
import json
import csv
import gzip
import re
import sys
from pathlib import Path

import pandas as pd

In [ ]:
PATH = Path("/Users/andrewprokhorenkov/CTDS/projects/midrc/wd/inventory/external-data-midrc-replication/external-data-midrc-replication-inventory-configuration-Parquet/")
PATH = PATH / "2022-06-26T00-00Z" / "manifest.json"

In [ ]:
with open(PATH) as manifest_data:
    manifest = json.load(manifest_data)

In [ ]:
all_files = []
tsvs = []

for obj in manifest['files']:
    print(obj)
    parquet_path = f"/Users/andrewprokhorenkov/CTDS/projects/midrc/wd/inventory/{obj['key']}"

    df = pd.read_parquet(parquet_path, engine="pyarrow")
    
    # print(df)

    all_files.extend(df.key)

    tsvs_df = df.key[df.key.str.contains(".tsv$")]

    tsvs.extend(tsvs_df.to_list())

In [ ]:
for f in all_files:
    if "1.2.826.0.1.3680043.10.474.419639.100534447940284283267949947795.dcm" in f:
        print(f)

In [ ]:
tsvs

In [ ]:
for f in tsvs:
    print(f"aws s3 cp s3://external-data-midrc-replication/{f} {f}")

In [ ]:
tsvs = []

for obj in manifest['files']:
    print(obj)
    gzip_obj = f"/Users/andrewprokhorenkov/CTDS/projects/midrc/wd/inventory/{obj['key']}"
    buffer = gzip.open(gzip_obj, mode='rt')
    reader = csv.reader(buffer)
    for row in reader:
        # print(row)
        # break
        filename = row[1]
        if filename[-4:] == ".dcm":
            tsvs.append(filename)

In [ ]:
submissions = []
debug_sub = []

acr_submission = re.compile(r".*?/([\w\d/-]*)/")
rsna_submission = re.compile(r".*?/([\w\d/-]*)/data")

for tsv in tsvs:
    if "replicated-data-acr" in tsv:
        match = acr_submission.match(tsv)
    elif "replicated-data-rsna" in tsv:
        match = rsna_submission.match(tsv)
    else:
        pass
        # print("That's bad...")

    debug_sub.append(match)
    if match:
        submission = match.group(1)
        if submission in submissions:
            continue
        submissions.append(submission)
    else:
        pass
        # print(tsv)

In [ ]:
for s in submissions:
    print(s)

In [ ]:
with open("/Users/andrewprokhorenkov/Downloads/listing_dicom_images.txt", "w") as download:
    for tsv in tsvs:
        download.write(tsv)
        download.write("\n")